In [6]:
# Taken from main06.cc Pythia example. A study of event properties of LEP1 events

from numpythia import Pythia, hepmc_write, hepmc_read
from numpythia import STATUS, HAS_END_VERTEX, ABS_PDG_ID

from pyjet import cluster, DTYPE_PTEPM, DTYPE_EP
from pyjet.testdata import get_event
from pyjet import ClusterSequenceArea, JetDefinition, ClusterSequence

import numpy as np
import sys

from dwave.system import EmbeddingComposite, DWaveSampler
import dimod

In [7]:
# Pythia generator settings. Process selection e+e- collision.
# WeakSingleBoson:ffbar2gmZ : Produce Z bosons and interference with photon
# Consider only Z (PDGID=23) decays to light quarks

params = {"Beams:idA" : "11", "Beams:idB" : "-11","Beams:eCM": 91.1876, "WeakSingleBoson:ffbar2gmZ" : "on", "23:onMode": "off", "23:onIfAny": "1 2 3 4 5", "PDF:lepton": "off"}
pythia = Pythia(params =params, random_state=1)

# Consider only final-state particles
selection = ((STATUS == 1) & ~HAS_END_VERTEX)

In [8]:
# Array of pseudo-jets or particles to cluster
particles = []
array = []

# Begin event loop. Generate events
for event in pythia(events=1):
    array = event.all(selection)

In [9]:
#For testing: filter out particles by energy
particles = array#[array["E"] > 2.0]

In [10]:
# Create qubo
from pyqubo import Spin, Array

# Create an array os spin variables
n_part = len(particles) 
s = Array.create('s', shape=n_part, vartype='BINARY')

# Create an array of zeroes for qubo matrix coefficients
coeff = [[0] * n_part for _ in range(n_part)] 
theta = np.pi/4.

for i in range(0,n_part):
    for j in range(0,n_part):
        coeff[i][j] = (particles[i]['px']*particles[j]['px'] + particles[i]['py']*particles[j]['py'] + particles[i]['pz']*particles[j]['pz'] - particles[i]['E']*particles[j]['E'] * np.cos(theta))/(1 - np.cos(theta))

# Construct Hamiltonian
H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

# Compile model using pyqubo

model = H.compile()
qubo, offset = model.to_qubo()

In [11]:
# Use default embedding on Advantage System
sampler = EmbeddingComposite(DWaveSampler(solver='Advantage_system1.1'))
sampleset = sampler.sample_qubo(qubo, return_embedding=True, num_reads=10, chain_strength=1.5)

In [12]:
# Convert qubo to bqm to visualize it in D-Wave's inspector
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo, offset = offset)

result = sampler.sample(bqm, num_reads=10)

In [13]:
import dwave.inspector

dwave.inspector.show(result)

ValueError: invalid combination of arguments provided: if data capture not enabled, problem/response/solver/etc have to be explicitly specified

In [ ]:
from chimera_embedding import processor
from minorminer import find_embedding

qpusampler = DWaveSampler(solver='Advantage_system1.1')
edges = qpusampler.structure.edgelist

In [ ]:
def make_complete_graph(size):
    jmatr = {}
    for i in range(size):
        for j in range(i+1, size):
            jmatr[(i,j)] = 1
    return jmatr

In [ ]:
def count_physical_qubits(embeding):
    nq = 0
    for sub_embed in embeding:
        nq += len(sub_embed)
    return nq

In [ ]:
gen_graph = make_complete_graph(len(particles))
embeddings = find_embedding(gen_graph, edges)

In [ ]:
embeddings

In [ ]:
import dwave.inspector

dwave.inspector.show(sampleset)